In [ ]:
! pip install exifread

In [ ]:
! pip list

In [ ]:
! pip install --upgrade tensorflow-hub

In [ ]:
! pip install "tensorflow<2.11" "numpy<2" tensorflow-gpu==2.10.0

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))


In [1]:
import tensorflow as tf

# 사용 가능한 GPU 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # 0번 GPU만 할당하고 이름 출력
    tf.config.set_visible_devices(gpus[0], 'GPU')
    print(f"NVIDIA GPU가 할당되었습니다: {gpus[0].device_type} - {gpus[0].name}")
else:
    print("사용 가능한 GPU가 없습니다.")


NVIDIA GPU가 할당되었습니다: GPU - /physical_device:GPU:0


In [4]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#----------------------------------------------------------------------
# 1) 경로 설정
#----------------------------------------------------------------------
base_dir = os.getcwd()  # 현재 작업 디렉토리
train_dir = os.path.join(base_dir, 'Train')
val_dir   = os.path.join(base_dir, 'Validation')
test_dir  = os.path.join(base_dir, 'Test')

#----------------------------------------------------------------------
# 2) 학습 파라미터 설정
#----------------------------------------------------------------------
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 16
EPOCHS = 10  # 필요에 따라 조정
NUM_CLASSES = 1  # Fake vs Real (이진분류)

#----------------------------------------------------------------------
# 3) ImageDataGenerator 설정
#    (데이터 증강 및 전처리)
#----------------------------------------------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

#----------------------------------------------------------------------
# 4) Directory로부터 이미지 로드 (flow_from_directory)
#----------------------------------------------------------------------
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',     # Fake / Real 이진 분류
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

#----------------------------------------------------------------------
# 5) 사전 학습된 EfficientNet-Lite 모델 불러오기 (TensorFlow Hub 예시)
#    - include_top=False 또는 feature-vector 형태 모델을 사용해 상위 레이어 재구성
#    - EfficientNet Lite0 ~ Lite4까지 다양한 버전이 있으므로, 아래 URL은 예시입니다.
#----------------------------------------------------------------------
efficientnet_lite_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"
feature_extractor = hub.KerasLayer(
    efficientnet_lite_url,
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    trainable=False  # 혹은 True로 설정하면 파라미터도 업데이트함(Full Fine-tuning)
)

#----------------------------------------------------------------------
# 6) 모델 구성
#----------------------------------------------------------------------
model = models.Sequential([
    feature_extractor,
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # 이진 분류이므로 sigmoid 사용
])

model.summary()  # 모델 구조 확인

#----------------------------------------------------------------------
# 7) 모델 컴파일
#----------------------------------------------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#----------------------------------------------------------------------
# 8) 모델 학습 (Fine-tuning)
#----------------------------------------------------------------------
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

#----------------------------------------------------------------------
# 9) 학습 결과 평가
#----------------------------------------------------------------------
print("---------- Validation Evaluation ----------")
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

print("---------- Test Evaluation ----------")
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

#----------------------------------------------------------------------
# 10) 모델 저장
#----------------------------------------------------------------------
model.save(os.path.join(base_dir, 'efficientnet_lite_finetuned.h5'))
print("Model saved!")


Found 6002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              3413024   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 3,414,305
Trainable params: 1,281
Non-trainable params: 3,413,024
_________________________________________________________________
Epoch 1/10
376/376 [==============================] - 41s 106ms/step - loss: 0.6863 - accuracy: 0.6038 - val_loss: 0.6299 - val_accuracy: 0.6524
Epoch 2/10
376/376 [===========

# ELA 학습

In [1]:
import os
from PIL import Image, ImageChops
import io

def error_level_analysis_pil(original_img, quality=70, scale=500):
    """
    원본 이미지를 ELA(Error Level Analysis) 후 반환
    :param original_img: PIL Image (RGB)
    :param quality: JPEG 재압축 품질
    :param scale: 차이를 시각적으로 증폭시키기 위한 스케일
    :return: ELA 결과를 PIL Image 형태로 반환
    """
    # 메모리에 임시로 JPEG 저장
    temp_io = io.BytesIO()
    original_img.save(temp_io, 'JPEG', quality=quality)
    temp_io.seek(0)

    # 재압축된 이미지 불러오기
    compressed_img = Image.open(temp_io).convert('RGB')

    # 원본과 재압축 이미지를 비교해 차이 계산
    diff = ImageChops.difference(original_img, compressed_img)

    # 차이를 원하는 배율(scale)만큼 곱해서 증폭
    diff = ImageChops.multiply(diff, Image.new('RGB', diff.size, (scale, scale, scale)))
    return diff

def convert_to_ela(original_dir, ela_dir, quality=70, scale=500, extensions=('.jpg', '.jpeg', '.png')):
    """
    원본 폴더(original_dir) 구조를 따라가며 모든 이미지를 ELA 변환 후,
    같은 구조의 ela_dir 폴더에 저장한다.
    
    :param original_dir: 원본 폴더 (예: Train/Real)
    :param ela_dir: ELA 변환된 이미지를 저장할 폴더 (예: ELA_Train/Real)
    :param quality: ELA에 사용될 JPEG 품질
    :param scale: ELA 차이 증폭 배율
    :param extensions: 처리할 이미지 확장자 목록
    """
    if not os.path.exists(ela_dir):
        os.makedirs(ela_dir, exist_ok=True)

    # 원본 폴더 내 파일/폴더를 순회
    for item in os.listdir(original_dir):
        src_path = os.path.join(original_dir, item)
        dst_path = os.path.join(ela_dir, item)

        if os.path.isdir(src_path):
            # 서브폴더인 경우, 재귀적으로 진행
            convert_to_ela(src_path, dst_path, quality, scale, extensions)
        else:
            # 파일인 경우
            ext = os.path.splitext(item)[1].lower()
            if ext in extensions:
                # 이미지 파일이면 ELA 적용 후 저장
                with Image.open(src_path).convert('RGB') as img:
                    ela_img = error_level_analysis_pil(img, quality=quality, scale=scale)
                    ela_img.save(dst_path)  # 확장자는 그대로, 혹은 .jpg로 통일 가능
            else:
                # 그 외 파일(예: txt)은 스킵하거나 복사
                pass

if __name__ == '__main__':
    # base_dir 설정
    base_dir = os.getcwd()

    # 원본 폴더 구조
    train_dir = os.path.join(base_dir, 'Train')
    val_dir   = os.path.join(base_dir, 'Validation')
    test_dir  = os.path.join(base_dir, 'Test')

    # ELA 저장할 폴더 구조
    ela_train_dir = os.path.join(base_dir, 'ELA_Train')
    ela_val_dir   = os.path.join(base_dir, 'ELA_Validation')
    ela_test_dir  = os.path.join(base_dir, 'ELA_Test')

    # 각 폴더를 순회하며 ELA 변환
    q = 80 # JPEG 품질
    s = 800 # ELA 차이 증폭 배율
    
    convert_to_ela(train_dir, ela_train_dir, quality=q, scale=s)
    convert_to_ela(val_dir, ela_val_dir, quality=q, scale=s)
    convert_to_ela(test_dir, ela_test_dir, quality=q, scale=s)

    print("ELA 변환 완료!")


ELA 변환 완료!


In [2]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#----------------------------------------------------------------------
# 1) 경로 설정
#----------------------------------------------------------------------
base_dir = os.getcwd()  # 현재 작업 디렉토리
train_dir = os.path.join(base_dir, 'ELA_Train')
val_dir = os.path.join(base_dir, 'ELA_Validation')
test_dir = os.path.join(base_dir, 'ELA_Test')

#----------------------------------------------------------------------
# 2) 학습 파라미터 설정
#----------------------------------------------------------------------
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 16
EPOCHS = 10  # 필요에 따라 조정
NUM_CLASSES = 1  # Fake vs Real (이진분류)

#----------------------------------------------------------------------
# 3) ImageDataGenerator 설정
#    (데이터 증강 및 전처리)
#----------------------------------------------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

#----------------------------------------------------------------------
# 4) Directory로부터 이미지 로드 (flow_from_directory)
#----------------------------------------------------------------------
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',     # Fake / Real 이진 분류
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

#----------------------------------------------------------------------
# 5) 사전 학습된 EfficientNet-Lite 모델 불러오기 (TensorFlow Hub 예시)
#    - include_top=False 또는 feature-vector 형태 모델을 사용해 상위 레이어 재구성
#    - EfficientNet Lite0 ~ Lite4까지 다양한 버전이 있으므로, 아래 URL은 예시입니다.
#----------------------------------------------------------------------
efficientnet_lite_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"
feature_extractor = hub.KerasLayer(
    efficientnet_lite_url,
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    trainable=False  # 혹은 True로 설정하면 파라미터도 업데이트함(Full Fine-tuning)
)

#----------------------------------------------------------------------
# 6) 모델 구성
#----------------------------------------------------------------------
model = models.Sequential([
    feature_extractor,
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # 이진 분류이므로 sigmoid 사용
])

model.summary()  # 모델 구조 확인

#----------------------------------------------------------------------
# 7) 모델 컴파일
#----------------------------------------------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#----------------------------------------------------------------------
# 8) 모델 학습 (Fine-tuning)
#----------------------------------------------------------------------
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

#----------------------------------------------------------------------
# 9) 학습 결과 평가
#----------------------------------------------------------------------
print("---------- Validation Evaluation ----------")
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

print("---------- Test Evaluation ----------")
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

#----------------------------------------------------------------------
# 10) 모델 저장
#----------------------------------------------------------------------
model.save(os.path.join(base_dir, 'efficientnet_lite_ela.h5'))
print("Model saved!")


Found 6002 images belonging to 2 classes.
Found 7324 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              3413024   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 3,414,305
Trainable params: 1,281
Non-trainable params: 3,413,024
_________________________________________________________________
Epoch 1/10
376/376 [==============================] - 112s 277ms/step - loss: 0.7001 - accuracy: 0.5045 - val_loss: 0.6885 - val_accuracy: 0.5168
Epoch 2/10
376/376 [=============

# 테스트 모델

In [2]:
import os
import tensorflow as tf
import tensorflow_hub as hub  # (사용하지 않아도 무방하나, 편의상 import)
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#----------------------------------------------------------------------
# 1) 경로 설정
#----------------------------------------------------------------------
base_dir = os.getcwd()  # 현재 작업 디렉토리
train_dir = os.path.join(base_dir, 'Train')
val_dir   = os.path.join(base_dir, 'Validation')
test_dir  = os.path.join(base_dir, 'Test')

#----------------------------------------------------------------------
# 2) 학습 파라미터 설정
#----------------------------------------------------------------------
IMAGE_SIZE = (64, 64)   # 이미지를 256x256으로 리사이즈
BATCH_SIZE = 16
EPOCHS = 10               # 필요에 따라 조정
NUM_CLASSES = 1           # Fake vs Real (이진분류)

#----------------------------------------------------------------------
# 3) ImageDataGenerator 설정 (데이터 증강 및 전처리)
#----------------------------------------------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

#----------------------------------------------------------------------
# 4) Directory로부터 이미지 로드 (flow_from_directory)
#----------------------------------------------------------------------
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Fake / Real 이진 분류
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

#----------------------------------------------------------------------
# 5) 모델 정의 (간단한 CNN)
#    - 입력: 256x256 RGB
#    - 출력: 이진 분류(sigmoid)
#----------------------------------------------------------------------
model = models.Sequential()

# 첫 번째 Conv2D 레이어
model.add(layers.Conv2D(16, (3, 3), activation='relu', 
                        padding='same', 
                        input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 두 번째 Conv2D 레이어
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 세 번째 Conv2D 레이어
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 네 번째 Conv2D 레이어
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 특징 맵을 1차원으로 펼쳐서 Dense에 전달
model.add(layers.Flatten())

# 완전 연결 레이어(Dense)
model.add(layers.Dense(32, activation='relu'))

# 출력 레이어: 이진 분류 → 1개의 노드 + sigmoid
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

#----------------------------------------------------------------------
# 6) 모델 컴파일
#----------------------------------------------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Found 6002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        9248 

In [3]:

#----------------------------------------------------------------------
# 7) 모델 학습
#----------------------------------------------------------------------
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

#----------------------------------------------------------------------
# 8) 학습 결과 평가
#----------------------------------------------------------------------
print("---------- Validation Evaluation ----------")
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

print("---------- Test Evaluation ----------")
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

#----------------------------------------------------------------------
# 9) 모델 저장
#----------------------------------------------------------------------
model.save(os.path.join(base_dir, 'custom_cnn_finetuned.h5'))
print("Model saved!")


Epoch 1/10
376/376 [==============================] - 50s 126ms/step - loss: 0.6756 - accuracy: 0.5956 - val_loss: 0.6859 - val_accuracy: 0.5657
Epoch 2/10
376/376 [==============================] - 29s 77ms/step - loss: 0.6193 - accuracy: 0.6753 - val_loss: 0.7460 - val_accuracy: 0.5801
Epoch 3/10
376/376 [==============================] - 28s 74ms/step - loss: 0.5771 - accuracy: 0.7079 - val_loss: 0.7086 - val_accuracy: 0.6080
Epoch 4/10
376/376 [==============================] - 28s 74ms/step - loss: 0.5508 - accuracy: 0.7271 - val_loss: 0.6434 - val_accuracy: 0.6457
Epoch 5/10
376/376 [==============================] - 28s 74ms/step - loss: 0.5397 - accuracy: 0.7358 - val_loss: 0.6685 - val_accuracy: 0.6285
Epoch 6/10
376/376 [==============================] - 28s 76ms/step - loss: 0.5286 - accuracy: 0.7426 - val_loss: 0.6704 - val_accuracy: 0.6409
Epoch 7/10
376/376 [==============================] - 28s 74ms/step - loss: 0.5164 - accuracy: 0.7532 - val_loss: 0.6800 - val_accuracy